In [2]:
import os
import json 
import requests
import pandas as pd
from bs4 import BeautifulSoup
from openai import OpenAI

In [3]:
# Load sheet metadata from sources.json
with open("sources.json", "r") as f:
    sources = json.load(f)

# Container for DataFrames
all_dfs = []

In [4]:
# Loop through each source and load the data
for source in sources:
    sheet_id = source["sheet_id"]
    gid = source["gid"]
    name = source["name"]

    # Construct URL
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&id={sheet_id}&gid={gid}"

    # Read CSV from the URL
    try:
        df = pd.read_csv(url)
        df['source'] = name  # Add a column to identify where it came from
        all_dfs.append(df)
        print(f"Successfully Loaded: {name}")
    except Exception as e:
        print(f"Failed to load data {name}: {e}")

# Concatenate all DataFrames
combined_df = pd.concat(all_dfs, ignore_index=True)

Successfully Loaded: Isfahan
Successfully Loaded: Fars
Successfully Loaded: Bushehr
Successfully Loaded: CMB
Successfully Loaded: Hormozgan
Successfully Loaded: KBA


In [5]:
combined_df

,Food_Name,Province,City,Meal_Type,Website,Pictures_Website,is_approved,source
0,بریانی,Isfahan,اصفهان,Food,https://chishi.ir/2768-beryani-esfehan/,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,Isfahan
1,https://hesetazegi.ir/fa/recipes/fr-tas-kabab-...,Isfahan,اصفهان,"Food, Appetizer",https://hesetazegi.ir/fa/recipes/fr-tas-kabab-...,https://hesetazegi.ir/_next/image?url=https%3A...,False,Isfahan
2,آماج اسفناج,Isfahan,اصفهان,"Food, Appetizer",https://hesetazegi.ir/fa/recipes/fr-amaj-spinach,https://hesetazegi.ir/_next/image?url=https%3A...,False,Isfahan
3,هلیم بادمجان,Isfahan,اصفهان,Food,https://hesetazegi.ir/fa/recipes/fr-eggplant-h...,https://hesetazegi.ir/_next/image?url=https%3A...,False,Isfahan
4,قیمه‌ریزه سیب‌زمینی,Isfahan,اصفهان,"Food, Appetizer",https://hesetazegi.ir/fa/recipes/fr-gheimeh-ri...,https://hesetazegi.ir/_next/image?url=https%3A...,False,Isfahan
...,...,...,...,...,...,...,...,...
331,دوا زردکی و پیاز داغی,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://www.sobhezagros.ir/story/%C2%AB%D8%B4%...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,KBA
332,اش دنگو,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://blog.okcs.com/ash-dango-recipe/,https://blog.okcs.com/wp-content/uploads/2022/...,True,KBA
333,آش کشک یاسوجی,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://sarashpazpapion.com/picture/2304310/%D...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,KBA
334,قورمه گوشت,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://chishi.ir/23820-ghorme-gousht/,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,KBA


In [6]:
# Copy all records to df for manipulation
df = combined_df.copy()
df["id"] = combined_df.index + 1

# Normalize column names to avoid casing issues
df.columns = [col.strip().lower() for col in df.columns]
# Check for availability of is_approved and website columns
if "is_approved" not in df.columns or "website" not in df.columns:
    raise ValueError("Your DataFrame must contain 'is_approved' and 'website' columns.")

# To copy all the approved records in a seperate variable
approved_df = df[df["is_approved"] == True].copy()
approved_df

,food_name,province,city,meal_type,website,pictures_website,is_approved,source,id
0,بریانی,Isfahan,اصفهان,Food,https://chishi.ir/2768-beryani-esfehan/,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,Isfahan,1
29,حلیم بادمجان,Isfahan,اصفهان,Food,https://namnak.com/%D8%AD%D9%84%DB%8C%D9%85-%D...,https://files.namnak.com/users/mh/aup/202108/3...,True,Isfahan,30
30,دوغ و گوشفیل,Isfahan,اصفهان,"Drinks, Dessert",https://mag.canbo.ir/recipe/gooshfil-and-doogh/,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,Isfahan,31
31,حلیم شیر,Isfahan,اصفهان,"Appetizer, Food",https://blog.okcs.com/halim-shir-recipe/,https://blog.okcs.com/wp-content/uploads/2024/...,True,Isfahan,32
32,کباب حسینی,Isfahan,اصفهان,Food,https://panamag.ir/6980-kabab-hosseini/,https://panamag.ir/wp-content/uploads/2023/01/...,True,Isfahan,33
...,...,...,...,...,...,...,...,...,...
331,دوا زردکی و پیاز داغی,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://www.sobhezagros.ir/story/%C2%AB%D8%B4%...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,KBA,332
332,اش دنگو,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://blog.okcs.com/ash-dango-recipe/,https://blog.okcs.com/wp-content/uploads/2022/...,True,KBA,333
333,آش کشک یاسوجی,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://sarashpazpapion.com/picture/2304310/%D...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,KBA,334
334,قورمه گوشت,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://chishi.ir/23820-ghorme-gousht/,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,KBA,335


In [12]:
approved_df["pictures_website"].head(1)

0    data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
Name: pictures_website, dtype: object

In [8]:
approved_df.to_csv("raw_data.csv")

In [11]:
# Make sure the 'foods' directory exists
os.makedirs("foods", exist_ok=True)

# Save the HTML content to a file named {id}.html inside 'foods' directory
def save_as_file(html_content, row_id):
    file_path = os.path.join("foods", f"{row_id}.html")
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(html_content)

In [12]:

html_records = []

for _, row in approved_df.iterrows():
    site_url = row["website"]
    row_id = row.get("id")  # make sure your dataframe has an 'id' column
    print(row_id)

    try:
        response = requests.get(site_url, timeout=5)
        html_content = response.text
        print(f"(website)->{site_url} — HTML length: {len(html_content)}")

        save_as_file(html_content=html_content, row_id= row_id)
        
        html_records.append({
            "id": row_id,
            "html": html_content
        })

    except requests.RequestException as e:
        print(f"Failed to fetch {site_url}: {e}")


1
(website)->https://chishi.ir/2768-beryani-esfehan/ — HTML length: 87708
30
(website)->https://namnak.com/%D8%AD%D9%84%DB%8C%D9%85-%D8%A8%D8%A7%D8%AF%D9%85%D8%AC%D8%A7%D9%86.p14908 — HTML length: 136715
31
(website)->https://mag.canbo.ir/recipe/gooshfil-and-doogh/ — HTML length: 57295
32
Failed to fetch https://blog.okcs.com/halim-shir-recipe/: HTTPSConnectionPool(host='blog.okcs.com', port=443): Max retries exceeded with url: /halim-shir-recipe/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
33
(website)->https://panamag.ir/6980-kabab-hosseini/ — HTML length: 66349
34
(website)->https://blog.okala.com/yakhni-lubia-isfahan/ — HTML length: 202501
35
Failed to fetch https://blog.okcs.com/meatballs-dill-beans-recipe/: HTTPSConnectionPool(host='blog.okcs.com', port=443): Max retries exceeded with url: /meatballs-dill-beans-recipe/ (Caused by SSLError(SSLCertVerificationEr

In [6]:
# Make sure Outputs directory exists
os.makedirs("Outputs", exist_ok=True)
        
def save_json(data, filename):
    with open(f"outputs/{filename}.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [7]:
# Load the base prompt
with open("data_extraction_prompt.txt", "r", encoding="utf-8") as file:
    base_prompt = file.read()

In [ ]:
# Set your OpenAI API key
client = OpenAI(api_key="api-key")

In [23]:
test_df = approved_df[approved_df["id"] > 159]
test_df

,food_name,province,city,meal_type,website,pictures_website,is_approved,source,id
159,تاس کباب,Chaharmahal and Bakhtiari,شهرکرد,Food,https://roostanet.ir/fa/6098,https://img.tebyan.net/big/1399/05/15224013172...,True,CMB,160
160,کباب بختیاری,Chaharmahal and Bakhtiari,شهرکرد,Food,https://www.offdecor.com/%D8%B7%D8%B1%D8%B2-%D...,https://www.offdecor.com/image/media/post/7080...,True,CMB,161
161,گوشت ترش بختیاری,Chaharmahal and Bakhtiari,شهرکرد,Food,https://telewebion.com/episode/0x1b5688c,https://telewebion.com/episode/0x1b5688c,True,CMB,162
162,آبگوشت ترشی,Chaharmahal and Bakhtiari,شهرکرد,Food,https://mamifood.org/cooking-training/recipe/5...,https://mamifood.org/images/files/x/photo_2019...,True,CMB,163
163,آبریز,Chaharmahal and Bakhtiari,شهرکرد,Food,https://www.delgarm.com/cooking/training-of-fo...,https://i1.delgarm.com/i/828/9906/23/5f5dab7a6...,True,CMB,164
...,...,...,...,...,...,...,...,...,...
331,دوا زردکی و پیاز داغی,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://www.sobhezagros.ir/story/%C2%AB%D8%B4%...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,KBA,332
332,اش دنگو,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://blog.okcs.com/ash-dango-recipe/,https://blog.okcs.com/wp-content/uploads/2022/...,True,KBA,333
333,آش کشک یاسوجی,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://sarashpazpapion.com/picture/2304310/%D...,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,KBA,334
334,قورمه گوشت,kohgiluyeh and boyer-ahmad,یاسوج,Food,https://chishi.ir/23820-ghorme-gousht/,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",True,KBA,335


In [24]:
log_path = "log.txt"
for idx, row in test_df.iterrows():
    food_name = row["food_name"].strip()
    food_id = row["id"]
    province = row["province"]
    city = row["city"]
    
    # Get the corresponding HTML record — adjust this line depending on how you're storing HTML
    html_dict = next((record for record in html_records if record["id"] == food_id), None)
    if html_dict is None:
        with open(log_path, "a", encoding="utf-8") as log_file:
            log_file.write(f"{food_id}, {food_name}, {province}, {city} | skipped (no HTML) ⏭️\n")
        continue
    html = html_dict["html"]
    # Parse HTML to extract body
    soup = BeautifulSoup(html, "html.parser")
    body_content = str(soup.body) if soup.body else ""
    
    # Replace placeholder with actual food name
    prompt_with_food = base_prompt.replace("{{FOOD_NAME}}", food_name)
    
    # Combine full prompt
    full_prompt = f"{prompt_with_food}\n\n{body_content}"
    
    try:
        # Call GPT API
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": full_prompt}],
            temperature=0.2
        )

        raw_response = response.choices[0].message.content
        cleaned_response = raw_response.strip('```json\n').strip('\n```')

        # Try to parse the response
        try:
            json_data = json.loads(cleaned_response)
            save_json(json_data, f"{food_id}_{food_name}_{province}")
            # ✅ Log success
            with open(log_path, "a", encoding="utf-8") as log_file:
                log_file.write(f"{food_id}, {food_name}, {province}, {city} | done ✅\n")
                
        except json.JSONDecodeError as parse_err:
            error_data = {
                "error": "JSON parsing failed",
                "message": str(parse_err),
                "raw_response": raw_response
            }
            save_json(error_data, f"{food_id}_{food_name}_{province}_parseFailed")
            # ❌ Log JSON parse failure
            with open(log_path, "a", encoding="utf-8") as log_file:
                log_file.write(f"{food_id}, {food_name}, {province}, {city} | parseFailed ❌\n")

    except Exception as api_err:
        error_data = {
            "error": "API call failed",
            "message": str(api_err)
        }
        save_json(error_data, f"{food_id}_{food_name}_{province}_apiFailed")
        # ❌ Log API failure
        with open(log_path, "a", encoding="utf-8") as log_file:
            log_file.write(f"{food_id}, {food_name}, {province}, {city} | apiFailed ❌\n")